In [10]:
import os 
import pytorch_lightning as pl
import seml
import numpy as np
import torch
from sacred import SETTINGS, Experiment
from functools import partial

import pandas as pd
import anndata

import scanpy as sc
import scvelo as scv
import cellrank as cr

from scCFM.datamodules.distribution_datamodule import TrajectoryDataModule
from scCFM.models.cfm.cfm_module import CFMLitModule
from scCFM.models.cfm.components.augmentation import AugmentationModule
from scCFM.models.cfm.components.simple_mlp import VelocityNet

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger

from torch.optim import AdamW

import yaml

import sys 
sys.path.insert(0, "../../../" )
from paths import EXPERIMENT_FOLDER

## Util functions 

In [2]:
eb_path = "/nfs/homedirs/pala/scCFM/project_dir/data/eb/eb_phate.h5ad"
hein_path = "/nfs/homedirs/pala/scCFM/project_dir/data/hein_et_al/processed/unperturbed_time_course_host_low.h5ad"
pancreas_path = "/nfs/homedirs/pala/scCFM/project_dir/data/pancreas/processed/pancreas.h5ad"
schiebinger_path = "/nfs/homedirs/pala/scCFM/project_dir/data/schiebinger_et_al/processed/schiebinger_et_al.h5ad"

In [3]:
path =  eb_path
x_layer = "X_log"
time_key = "experimental_time" 
use_pca = False
n_dimensions = None 
train_val_test_split = [0.8, 0.2]
batch_size = 3000
num_workers = 0
seed = 1234

Initialization

In [4]:
# Initialize datamodule
datamodule = TrajectoryDataModule(path = path,
                                    x_layer = x_layer,
                                    time_key = time_key, 
                                    use_pca = use_pca, 
                                    n_dimensions = n_dimensions, 
                                    train_val_test_split = train_val_test_split,
                                    batch_size = batch_size,
                                    num_workers = num_workers)

/nfs/staff-ssd/pala/envs/scCFM/lib/python3.9/site-packages/anndata/_core/anndata.py:1830: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [5]:
training_set = datamodule.train_dataloader(load_full=True)

In [6]:
for i in next(iter(training_set)):
    print(i.shape)

torch.Size([3000, 1579])
torch.Size([3000, 1579])
torch.Size([3000, 1579])
torch.Size([3000, 1579])
torch.Size([3000, 1579])


In [7]:
next(iter(training_set))

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 1.1845, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
     

In [8]:
torch.zeros(1)

tensor([0.])

In [9]:
datamodule.idx2time

{0: 0.0, 1: 0.25, 2: 0.5, 3: 0.75, 4: 1.0}